<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prepare-the-data" data-toc-modified-id="Prepare-the-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prepare the data</a></span><ul class="toc-item"><li><span><a href="#Raw-data" data-toc-modified-id="Raw-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Raw data</a></span><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Extract-the-unit-floor" data-toc-modified-id="Extract-the-unit-floor-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Extract the unit floor</a></span></li><li><span><a href="#Keep-only-the-relevant-data" data-toc-modified-id="Keep-only-the-relevant-data-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Keep only the relevant data</a></span></li><li><span><a href="#Create-a-'transaction_year'-column" data-toc-modified-id="Create-a-'transaction_year'-column-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>Create a 'transaction_year' column</a></span></li><li><span><a href="#Convert-to-datetime" data-toc-modified-id="Convert-to-datetime-1.1.5"><span class="toc-item-num">1.1.5&nbsp;&nbsp;</span>Convert to datetime</a></span></li><li><span><a href="#Convert-string-column-region-to-lowercase" data-toc-modified-id="Convert-string-column-region-to-lowercase-1.1.6"><span class="toc-item-num">1.1.6&nbsp;&nbsp;</span>Convert string column region to lowercase</a></span></li><li><span><a href="#Correct-typo-on-the-district-name-'Hung-Hum'" data-toc-modified-id="Correct-typo-on-the-district-name-'Hung-Hum'-1.1.7"><span class="toc-item-num">1.1.7&nbsp;&nbsp;</span>Correct typo on the district name 'Hung Hum'</a></span></li><li><span><a href="#Fill-the-empty-cells" data-toc-modified-id="Fill-the-empty-cells-1.1.8"><span class="toc-item-num">1.1.8&nbsp;&nbsp;</span>Fill the empty cells</a></span></li><li><span><a href="#Calculate-the-netPricePerSqf,-resampled-and-normalized" data-toc-modified-id="Calculate-the-netPricePerSqf,-resampled-and-normalized-1.1.9"><span class="toc-item-num">1.1.9&nbsp;&nbsp;</span>Calculate the netPricePerSqf, resampled and normalized</a></span></li><li><span><a href="#Check-the-output" data-toc-modified-id="Check-the-output-1.1.10"><span class="toc-item-num">1.1.10&nbsp;&nbsp;</span>Check the output</a></span></li></ul></li><li><span><a href="#Supplementary-data" data-toc-modified-id="Supplementary-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Supplementary data</a></span><ul class="toc-item"><li><span><a href="#Hong-Kong-regions" data-toc-modified-id="Hong-Kong-regions-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Hong Kong regions</a></span></li><li><span><a href="#HANG-SENG-INDEX" data-toc-modified-id="HANG-SENG-INDEX-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>HANG SENG INDEX</a></span></li><li><span><a href="#Hong-Kong-GDP" data-toc-modified-id="Hong-Kong-GDP-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Hong Kong GDP</a></span></li></ul></li><li><span><a href="#Export-the-processed-data" data-toc-modified-id="Export-the-processed-data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Export the processed data</a></span><ul class="toc-item"><li><span><a href="#For-the-visualizations:-join-the-3-datasets" data-toc-modified-id="For-the-visualizations:-join-the-3-datasets-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>For the visualizations: join the 3 datasets</a></span><ul class="toc-item"><li><span><a href="#Absolute-values" data-toc-modified-id="Absolute-values-1.3.1.1"><span class="toc-item-num">1.3.1.1&nbsp;&nbsp;</span>Absolute values</a></span></li><li><span><a href="#Normalized-data,-base-100" data-toc-modified-id="Normalized-data,-base-100-1.3.1.2"><span class="toc-item-num">1.3.1.2&nbsp;&nbsp;</span>Normalized data, base 100</a></span></li></ul></li><li><span><a href="#For-modeling" data-toc-modified-id="For-modeling-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>For modeling</a></span></li></ul></li></ul></li></ul></div>

In [1]:
# Import libraries necessary for this project
import sqlite3
import numpy as np
import pandas as pd
from pandas_datareader import data
import os
import datetime
import re

# Prepare the data

## Raw data

### Load

In [2]:
#connect and load the database
#online source: https://centadata.publicdata.hk/centadata-bd4a1e3.db
centadata = sqlite3.connect('data/raw/centadata-2018-11-19.db')
data = pd.read_sql_query("select * from centadatatransactions join centadataunits using (unitcode) join centadatabuildings using (buildingcode) WHERE txdate BETWEEN '1996-01-01' AND '2017-12-31'", centadata)

In [3]:
# Load the Hong Kong housing dataset
# data = pd.read_csv('data/raw/dataset-10-01-2019.csv')
print("data len: ",len(data))
print(data['txdate'].min(), data['txdate'].max())

data len:  1815620
1996-01-02 2017-12-29


In [4]:
data['price'].describe().apply(lambda x: format(x, 'f'))

count       1815620.000000
mean        3972486.298756
std         6475576.255079
min           10000.000000
25%         1510000.000000
50%         2510000.000000
75%         4380000.000000
max      1020000000.000000
Name: price, dtype: object

### Extract the unit floor

In [5]:
data["floor"] = data["yaxis"].str.extract('(?i)(\d+|g)\w?(?=\/f)', expand=False)
data['floor'] = data['floor'].replace('G','0')
data['floor'] = data['floor'].fillna('0')
data['floor'] = data['floor'].astype('int')
print("after cleanup: " + str(len(data[data['floor'].isna()])))

after cleanup: 0


In [6]:
data["floor"].unique()

array([ 1,  0,  2,  3,  5,  8,  4,  6,  7,  9, 10, 11, 12, 13, 15, 16, 17,
       18, 19, 14, 20, 21, 23, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       35, 36, 39, 37, 38, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 34, 44,
       60, 51, 52, 53, 55, 56, 57, 58, 59, 54, 66, 61, 62, 63, 64, 65, 67,
       68, 69, 70, 71, 72, 75, 76, 73, 77, 79, 78, 80, 87, 88, 89, 90, 81,
       82, 83, 85, 86], dtype=int64)

In [7]:
# declare the relevant period for the other datasets
date_from = '1996-01-01'
date_until = '2017-12-31'data['yearofcompletion'] = data['dateofcompletion'].str.extract('(\d{4})', expand=False)

### Keep only the relevant data

In [8]:
# # declare the relevant period for the other datasets
# date_from = '1996-01-01'
# date_until = '2017-12-31'

# # filter the dataset
# data = data[data['year'] >= int(re.match('\d{4}', date_from).group())]
# data = data[data['year'] <= int(re.match('\d{4}', date_until).group())]

# for testing let's limit the size of the df
# data = data.sample(100000)

len(data)

1815620

In [9]:
cols = ['unitcode',
 'buildingcode',
 'xaxis',
 'yaxis',
 'buildingname',
 'phasecode',
 'phasename',
'dateofcompletion',
'numberofunits',
'numberoffloors',
'flatsperfloor',
 'estatecode',
 'estatename',
 'buildingaddress',
'schoolnet',
'buildingnamechinese',
'phasenamechinese',
'estatenamechinese',
'buildingaddresschinese',
'regionchinese',
'propertydeveloperchinese',
'propertyfacilitieschinese',
'propertymanagerchinese'
]
data.drop(cols, axis=1, inplace=True)
data = data.sort_values('txdate')

In [10]:
# The most expensive transaction !
data.sort_values('price').tail(1)

,txdate,price,saleablearea,grossarea,region,regioncode,floor,yearofcompletion
561203,2016-02-16,1020000000,11937.0,NaN,Peak/South,107,0,1966


In [11]:
data.columns

Index(['txdate', 'price', 'saleablearea', 'grossarea', 'region', 'regioncode',
       'floor', 'yearofcompletion'],
      dtype='object')

In [12]:
len(data.regioncode.unique())

56

In [13]:
len(data.region.unique())

57

In [14]:
data.dtypes

txdate               object
price                 int64
saleablearea        float64
grossarea           float64
region               object
regioncode            int64
floor                 int32
yearofcompletion     object
dtype: object

### Create a 'transaction_year' column

In [15]:
data['transaction_year'] = data['txdate'].apply(lambda x: str(x).split('-')[0]).astype(str).astype(int)

### Convert to datetime

In [16]:
data['txdate'] = pd.to_datetime(data['txdate'], format="%Y-%m-%d")
data.rename(columns={"txdate": "date"},inplace=True)
data.set_index('date',inplace=True,drop=True)

### Convert string column region to lowercase

In [17]:
data['region'] = data['region'].apply(lambda x: x.lower() if isinstance(x, str) else x)

### Correct typo on the district name 'Hung Hum'

In [18]:
data['region'] = data['region'].replace('Hung Hum','Hung Hom')

### Fill the empty cells

In [19]:
print("remove the transactions that have neither the net or gross area")
print("empty values before cleanup: " + str(len(data[(data['saleablearea'].isna() == True) & (data['grossarea'].isna() == True)])))

# remove these transactions from the dataset
data = data[(data['saleablearea'].isna() == False) | (data['grossarea'].isna() == False)]
print("empty values after cleanup: " +  str(len(data[(data['saleablearea'].isna() == True) & (data['grossarea'].isna() == True)])))

remove the transactions that have neither the net or gross area
empty values before cleanup: 51326
empty values after cleanup: 0


In [20]:
dfRatio = data['saleablearea'][(data['saleablearea'].isna() == False) & (data['grossarea'].isna() == False)] / data['grossarea'][(data['saleablearea'].isna() == False) & (data['grossarea'].isna() == False)]
dfRatio.mean()

0.7698564951500763

In [21]:
print("Calculating the value of saleablearea based on the average net/gross ratio")
print("empty values before cleanup: " + str(len(data[data['saleablearea'].isna() == True])))
data['saleablearea'] = data['saleablearea'].fillna(data['grossarea'] * dfRatio.mean())
data['saleablearea'] = data['saleablearea'].astype("int")
print("empty values after cleanup: " + str(len(data[data['saleablearea'].isna() == True])))

Calculating the value of saleablearea based on the average net/gross ratio
empty values before cleanup: 335263
empty values after cleanup: 0


In [22]:
print("Calculating the value of grossarea based on the average net/gross ratio")
print("empty values before cleanup: " + str(len(data[data['grossarea'].isna() == True])))
data['grossarea'] = data['grossarea'].fillna(data['saleablearea'] / dfRatio.mean())
data['grossarea'] = data['grossarea'].astype("int")
print("empty values after cleanup: " + str(len(data[data['grossarea'].isna() == True])))

Calculating the value of grossarea based on the average net/gross ratio
empty values before cleanup: 180260
empty values after cleanup: 0


In [23]:
print("remove the transactions that have no date of completion")
print("empty values before cleanup: " + str(len(data[(data['yearofcompletion'].isna() == True)])))

# remove these transactions from the dataset
data = data[(data['yearofcompletion'].isna() == False)]
print("empty values after cleanup: " +  str(len(data[(data['yearofcompletion'].isna() == True)])))

remove the transactions that have no date of completion
empty values before cleanup: 10533
empty values after cleanup: 0


### Calculate the netPricePerSqf, resampled and normalized

In [24]:
data['netPricePerSqf'] = data['price'] / data['saleablearea']
data['netPricePerSqf'] = data['netPricePerSqf'].astype('int')
netPricePerSqf  = data['netPricePerSqf']

In [25]:
# Resampled on the first day of the month
netPricePerSqf_resampled_MS = data.resample('MS').median()

# Normalized, Resampled on the first day of the month
normalized_netPricePerSqf_MS = netPricePerSqf_resampled_MS.div(netPricePerSqf_resampled_MS.iloc[0]).mul(100)
# normalized_netPricePerSqf_MS = normalized_netPricePerSqf_MS.to_frame()
# normalized_netPricePerSqf_MS.columns = ['Hong Kong Property net price per SQF']

In [26]:
# Resampled on the first day of the quarter
netPricePerSqf_resampled_QS = data['netPricePerSqf'].resample('QS').median()
netPricePerSqf_resampled_QS = netPricePerSqf_resampled_QS.to_frame()

# Normalized, Resampled on the first day of the quarter
normalized_netPricePerSqf_QS = netPricePerSqf_resampled_QS.div(netPricePerSqf_resampled_QS.iloc[0]).mul(100)

### Check the output

In [27]:
data_processed = data

# Success
print("The processed transaction dataset has {} data points with {} variables each.\n".format(*data.shape))
print("empty cells:\n" + str(data.isnull().sum()))
data_processed.sample(5)

The processed transaction dataset has 1753761 data points with 9 variables each.

empty cells:
price               0
saleablearea        0
grossarea           0
region              0
regioncode          0
floor               0
yearofcompletion    0
transaction_year    0
netPricePerSqf      0
dtype: int64


,price,saleablearea,grossarea,region,regioncode,floor,yearofcompletion,transaction_year,netPricePerSqf
date,,,,,,,,,
2003-08-21,1430000,434,564,north point/fortress hill,112,19,1993,2003,3294
2012-10-30,5040000,477,620,tuen mun,402,33,2012,2012,10566
2000-08-07,2490000,528,687,tsing yi,406,8,2000,2000,4715
2014-02-05,3600000,295,383,sheung wan/central/admiralty,117,21,1985,2014,12203
1999-07-20,810000,535,695,tuen mun,402,10,1999,1999,1514


In [28]:
# Minimum price of the data
minimum_price = np.amin(data.price)

# Maximum price of the data
maximum_price = np.amax(data.price)

# Mean price of the data
mean_price = int(np.mean(data.price))

# Median price of the data
median_price = np.median(data.price)

# Standard deviation of prices of the data
std_price = int(np.std(data.price))

# Show the calculated statistics
print("Statistics for the Hong Kong housing dataset:\n")
print("Number of transactions in the dataset: " + str(len(data['netPricePerSqf'])))
print("\nTransactions:\nMinimum price: {} HKD".format(minimum_price)) 
print("Maximum price: {} HKD".format(maximum_price))
print("Mean price: {} HKD".format(mean_price))
print("Median price {} HKD".format(median_price))
print("Standard deviation of prices: {} HKD".format(std_price))

# Minimum price of the data
minimum_netPricePerSqf = np.amin(netPricePerSqf)

# Maximum price of the data
maximum_netPricePerSqf = np.amax(netPricePerSqf)

# Mean price of the data
mean_netPricePerSqf = int(np.mean(netPricePerSqf))

# Median price of the data
median_netPricePerSqf = np.median(netPricePerSqf)

# Standard deviation of prices of the data
std_netPricePerSqf = int(np.std(netPricePerSqf))

# Show the calculated statistics
print("\nPrice per SQF:\nMinimum net price per SQF: {0:.0f} HKD".format(minimum_netPricePerSqf)) 
print("Maximum net price per SQF: {0:.0f} HKD".format(maximum_netPricePerSqf))
print("Mean net price per SQF: {0:.0f} HKD".format(mean_netPricePerSqf))
print("Median net price per SQF: {0:.0f} HKD".format(median_netPricePerSqf))
print("Standard deviation of net price per SQF: {0:.0f} HKD".format(std_netPricePerSqf))

Statistics for the Hong Kong housing dataset:

Number of transactions in the dataset: 1753761

Transactions:
Minimum price: 10000 HKD
Maximum price: 1020000000 HKD
Mean price: 4017230 HKD
Median price 2560000.0 HKD
Standard deviation of prices: 6421162 HKD

Price per SQF:
Minimum net price per SQF: 14 HKD
Maximum net price per SQF: 1227450 HKD
Mean net price per SQF: 6315 HKD
Median net price per SQF: 5167 HKD
Standard deviation of net price per SQF: 4444 HKD


## Supplementary data

### Hong Kong regions

In [29]:
# Matching regions and district
# url="https://docs.google.com/spreadsheets/d/e/2PACX-1vRiiJ1hywSJcovAGQslTkI6g6aIQRc_uVe3-bE8Lee_b68MWVE3AWjtf5y1wwwdf-OWstIA7HqNcjme/pub?gid=0&single=true&output=csv"
df_district = pd.read_csv('data/external/Hong_Kong_districts - Sheet1.csv')

In [30]:
data_processed_region = data_processed.reset_index().merge(df_district).set_index('date')
print("empty cells:\n" + str(data_processed_region.isnull().sum()))

empty cells:
price               0
saleablearea        0
grossarea           0
region              0
regioncode          0
floor               0
yearofcompletion    0
transaction_year    0
netPricePerSqf      0
district            0
dtype: int64


In [31]:
data_processed_region.head()

,price,saleablearea,grossarea,region,regioncode,floor,yearofcompletion,transaction_year,netPricePerSqf,district
date,,,,,,,,,,
1996-01-02,2240000,381,487,to kwa wan,206,7,1995,1996,5879,Kowloon
1996-01-02,2230000,379,486,to kwa wan,206,7,1995,1996,5883,Kowloon
1996-01-02,2360000,374,487,to kwa wan,206,21,1995,1996,6310,Kowloon
1996-01-02,2860000,470,611,to kwa wan,206,15,1995,1996,6085,Kowloon
1996-01-02,3000000,470,611,to kwa wan,206,13,1995,1996,6382,Kowloon


### HANG SENG INDEX

In [32]:
# HANG SENG INDEX (^HSI) Yahoo Finance - Currency in HKD - Time Period: Dec 31, 1986 - Apr 10, 2019
hang_seng = pd.read_csv('data/external/finance-yahoo-HSI-daily-max.csv')
hang_seng.drop(columns=['High', 'Low', 'Close', 'Adj Close', 'Volume'],inplace=True)
hang_seng.rename(columns = {'Open':'hang-seng-index'},inplace=True)
hang_seng.rename(columns = {'Date': 'date'},inplace=True)

hang_seng['date'] = pd.to_datetime(hang_seng['date'], format="%Y-%m-%d")
hang_seng.set_index('date',inplace=True,drop=True)

hang_seng = hang_seng.resample('D').median()

# propagate the last valid observation forward to next valid
# hang_seng.fillna(method='ffill',inplace=True)
hang_seng['hang-seng-index'].interpolate(method='time',inplace=True)
hang_seng['hang-seng-index'] = hang_seng['hang-seng-index'].astype('int')

In [33]:
hang_seng = hang_seng[hang_seng.index >= date_from]
hang_seng = hang_seng[hang_seng.index <= date_until]
print("len: " + str(len(hang_seng)))
print(hang_seng.head())

len: 8036
            hang-seng-index
date                       
1996-01-01            10052
1996-01-02            10070
1996-01-03            10252
1996-01-04            10506
1996-01-05            10470


In [34]:
hang_seng.isnull().sum()

hang-seng-index    0
dtype: int64

In [35]:
hang_seng.describe()

,hang-seng-index
count,8036.000000
mean,17448.503733
std,5491.848569
min,6649.000000
25%,12684.500000
50%,17122.500000
75%,22298.250000
max,31783.000000


In [36]:
hang_seng_resampled_QS = hang_seng.resample('QS').median()
normalized_hang_seng = hang_seng_resampled_QS.div(hang_seng_resampled_QS.iloc[0]).mul(100)

### Hong Kong GDP

In [37]:
# Hong Kong GDP
# url="https://www.censtatd.gov.hk/showtablenewexcel.jsp?tableID=030&charsetID=1"
# HK_GDP = pd.read_excel(url,header=4,usecols=[0,2],)

HK_GDP = pd.read_excel('data/external/hong-kong-GDP.xls',header=4,usecols=[0,1,2])

In [38]:
# Quartely data
HK_GDP['Quarter'].replace(to_replace=r'\s[rp]', value='', regex=True,inplace=True)
HK_GDP = HK_GDP[(HK_GDP.Quarter == 'Q1') | (HK_GDP.Quarter == 'Q2')| (HK_GDP.Quarter == 'Q3')| (HK_GDP.Quarter == 'Q4')]
HK_GDP['Year'].fillna(method='ffill',inplace=True)
HK_GDP['Quarter'].replace(to_replace=r'Q1', value='-01-01', regex=True,inplace=True)
HK_GDP['Quarter'].replace(to_replace=r'Q2', value='-04-01', regex=True,inplace=True)
HK_GDP['Quarter'].replace(to_replace=r'Q3', value='-07-01', regex=True,inplace=True)
HK_GDP['Quarter'].replace(to_replace=r'Q4', value='-10-01', regex=True,inplace=True)
HK_GDP["date"] = HK_GDP["Year"].map(str) + HK_GDP["Quarter"]
HK_GDP = HK_GDP[['date','HK$ million']]
HK_GDP['date'] = pd.to_datetime(HK_GDP['date'], format="%Y-%m-%d")
HK_GDP.set_index('date',inplace=True)

In [39]:
HK_GDP.rename(columns = {'HK$ million':'hong-kong-gdp'}, inplace=True)
HK_GDP['hong-kong-gdp'] = HK_GDP['hong-kong-gdp'].astype('int')
HK_GDP = HK_GDP.resample('D').median()
HK_GDP.interpolate(method='time',inplace=True)
HK_GDP = HK_GDP.astype('int')

In [40]:
HK_GDP = HK_GDP[HK_GDP.index >= date_from]
HK_GDP = HK_GDP[HK_GDP.index <= date_until]
print("len: " + str(len(HK_GDP)))
HK_GDP.head()

len: 8036


,hong-kong-gdp
date,
1996-01-01,279132
1996-01-02,279339
1996-01-03,279547
1996-01-04,279755
1996-01-05,279962


In [41]:
HK_GDP_resampled_QS = HK_GDP.resample('QS').median()
normalized_HK_GDP = HK_GDP_resampled_QS.div(HK_GDP_resampled_QS.iloc[0]).mul(100)

## Export the processed data

### For the visualizations: join the 3 datasets

In [42]:
hong_kong_property_prices_all = data_processed_region.join(hang_seng, how='left').join(HK_GDP, how='left')

In [43]:
# Success
print("The processed transaction dataset has {} data points with {} variables each.\n".format(*hong_kong_property_prices_all.shape))
print("empty cells:\n" + str(hong_kong_property_prices_all.isnull().sum()))
hong_kong_property_prices_all.sample(5)

The processed transaction dataset has 1753761 data points with 12 variables each.

empty cells:
price               0
saleablearea        0
grossarea           0
region              0
regioncode          0
floor               0
yearofcompletion    0
transaction_year    0
netPricePerSqf      0
district            0
hang-seng-index     0
hong-kong-gdp       0
dtype: int64


,price,saleablearea,grossarea,region,regioncode,floor,yearofcompletion,transaction_year,netPricePerSqf,district,hang-seng-index,hong-kong-gdp
date,,,,,,,,,,,,
2004-01-07,1220000,554,712,tsing yi,406,7,1988,2004,2202,New Territories,13122,314026
2010-02-10,4950000,638,876,tseung kwan o,209,13,2001,2010,7758,New Territories,19914,418331
2003-11-12,2040000,865,1124,tung chung/islands,408,12,2002,2003,2358,New Territories,12021,325766
2007-10-25,1250000,304,384,shau kei wan/chai wan,115,9,1982,2007,4111,Hong Kong Island,29660,448407
2014-11-20,3280000,221,347,mongkok/yaumatei,203,23,2004,2014,14841,Kowloon,23401,592681


In [44]:
print("The processed transaction dataset has {} data points with {} variables each.\n".format(*data.shape))
print("empty cells:\n" + str(data.isnull().sum()))
hong_kong_property_prices_all.sample(5)

The processed transaction dataset has 1753761 data points with 9 variables each.

empty cells:
price               0
saleablearea        0
grossarea           0
region              0
regioncode          0
floor               0
yearofcompletion    0
transaction_year    0
netPricePerSqf      0
dtype: int64


,price,saleablearea,grossarea,region,regioncode,floor,yearofcompletion,transaction_year,netPricePerSqf,district,hang-seng-index,hong-kong-gdp
date,,,,,,,,,,,,
2005-05-30,1350000,284,395,shatin,302,30,1985,2005,4753,New Territories,13741,354992
1997-05-09,8650000,762,991,mid level west,106,11,1994,1997,11351,Hong Kong Island,13879,348213
2014-11-05,6650000,489,716,sheung wan/central/admiralty,117,7,2003,2014,13599,Hong Kong Island,23846,600010
1996-10-04,2850000,869,1130,kennedy town/sai yin pun,101,4,1966,1996,3279,Hong Kong Island,11985,337429
1999-06-10,5280000,794,983,kowloon station,202,25,2000,1999,6649,Kowloon,13022,326091


#### Absolute values

In [45]:
hong_kong_property_prices_all.columns

Index(['price', 'saleablearea', 'grossarea', 'region', 'regioncode', 'floor',
       'yearofcompletion', 'transaction_year', 'netPricePerSqf', 'district',
       'hang-seng-index', 'hong-kong-gdp'],
      dtype='object')

In [46]:
hong_kong_property_prices_HSI_GDP_district = hong_kong_property_prices_all[['price','transaction_year','saleablearea','district','yearofcompletion',
       'netPricePerSqf','hang-seng-index','hong-kong-gdp']]
hong_kong_property_prices_HSI_GDP_district.to_csv('data/processed/hong-kong-property-prices-HSI-GDP-district.csv')

#### Normalized data, base 100

In [47]:
hong_kong_property_prices_HSI_GDP_normalized = normalized_netPricePerSqf_QS.join(normalized_hang_seng, how='left').join(normalized_HK_GDP, how='left')
hong_kong_property_prices_HSI_GDP_normalized = hong_kong_property_prices_HSI_GDP_normalized.round(0)
hong_kong_property_prices_HSI_GDP_normalized.to_csv('data/processed/hong-kong-property-prices-HSI-GDP-normalized.csv')
hong_kong_property_prices_HSI_GDP_normalized.head()

,netPricePerSqf,hang-seng-index,hong-kong-gdp
date,,,
1996-01-01,100.0,100.0,100.0
1996-04-01,107.0,99.0,107.0
1996-07-01,111.0,101.0,114.0
1996-10-01,119.0,116.0,113.0
1997-01-01,130.0,121.0,113.0


### For modeling

In [48]:
#Select the regions with over 5000 transactions
data_model = data[data.groupby("region")['region'].transform('size') > 5000]

# Select the most homogeneous regions
data_model = data_model[data_model.groupby('region')['netPricePerSqf'].transform('var')  < 8e+06]

data_model.drop(columns=['region'],inplace=True)

In [49]:
len(data_model)

634069

In [50]:
data_model.head()

,price,saleablearea,grossarea,regioncode,floor,yearofcompletion,transaction_year,netPricePerSqf
date,,,,,,,,
1996-01-02,2200000,561,823,409,12,1986,1996,3921
1996-01-02,1190000,501,620,402,36,1990,1996,2375
1996-01-02,1790000,584,743,402,10,1993,1996,3065
1996-01-02,3130000,761,972,308,1,1992,1996,4113
1996-01-02,2250000,447,562,302,30,1986,1996,5033


In [51]:
data_model.to_csv('data/processed/hong-kong-property-prices-model.csv')